**SCHEMA**

NAZIONE_TEMPO(**Ccode**,**Anno**,Nome,Continente, Nome_Area_geografica ) 

POPOLAZIONE(**Ccode**,**Anno**, Aspettativa_di_vita, Numero_popolazione, Età_mediana, %persone_dipendono_dai_lavoratori, PILprocapite, Numero_medio_figli, %bambini_dipendono_dai_lavoratori, Rate_di_crescita)

AREA_GEOGRAFICA(**Nome_Area_Geografica**, Numero_morti_confilitti)

FATTORI_RILEVANTI(**Percentuale_sopravvivenza_fino_65_anni**, Numero_morti_per_cancro, Numero_morti_per_malattie_cardio, Numero_morti_infantili, Numero_morti_per_incidenti_stradali, Numero_morti_per_disastri_naturali, Numero_morti_per_suicidio, Numero_morti_per_fumo, Numero_morti_per_HIV, Numero_morti_per_fame, Numero_morti_per_tubercolosi, Numero_morti_per_demenza_senile, Numero_morti_per_annegamento,Numeri_morti_per_malaria, Ccode)

**RECUPERA I DATI CSV**







Link di partenza https://ourworldindata.org/age-structure 

 Tabelle *life-expectancy-vs-gdp-per-capita.csv* , *men-survival-to-age-65.csv*, w*omen-survival-to-age-65.csv* da https://ourworldindata.org/life-expectancy

Tabelle *cancer-death-rates.csv, cardiovascular-disease-death-rates.csv, death-rate-smoking.csv, death-rates-from-disasters.csv, death-rates-road-incidents.csv,dementia-death-rates.csv, diarrheal-disease-death-rates.csv, drowning-death-rates.csv, hiv-death-rates.csv , intentional-homicides-per-100000-people.csv, malaria-death-rates.csv,malnutrition-death-rates.csv,suicide-death-rates.csv, tuberculosis-death-rates.csv* da https://ourworldindata.org/causes-of-death

Tabella *child-mortality-igme.csv* da https://ourworldindata.org/child-mortality 

Tabella *conflitti/battle-related-deaths-in-state-based-conflicts-since-1946-by-world-region.csv* da https://ourworldindata.org/war-and-peace

Tabella *continents/country-and-continent-codes-list-csv.csv* da https://gist.github.com/stevewithington/20a69c0b6d2ff846ea5d35e5fc47f26c#file-country-and-continent-codes-list-csv-csv





**LETTURA DATI DAI CSV**

In [1]:
# creazione della tabella NAZIONE_TEMPO
import pandas as pd 
import numpy as np

# ho preso una tabella con le nazioni e gli anni per generare la tabella NAZIONE_TEMPO
N = pd.read_csv("../input/age-structure/median-age-vs-births-per-woman (2).csv")

# ho tolto certi attributi che non dovevano essere nella tabella NAZIONE_TEMPO
N.columns = ["Nome", "Ccode", "Anno", "Eta_media","Fertilita","Popolazione", "Continente"]
N = N[["Nome", "Ccode", "Anno", "Continente"]]

# ho fatto un gruppo di regioni che non sono nazioni 
series = set(N["Nome"])
not_countries = ["Antarctica","Oceania", "Northern America","Low-income countries", "Latin America and the Caribbean"'Least developed countries',"Land-locked Developing Countries (LLDC)","Lower-middle-income countries","Lower-income countries","Small Island Developing States (SIDS)","Western Sahara","Upper-middle-income countries",'Eastern Mediterranean Region (WHO)','Less developed regions, excluding China',"High-income countries","Less developed regions, excluding least developed countries","World","Less developed regions","Two Sicilies","East Germany","North America (WB)","European Region (WHO)", 'World Bank Lower Middle Income', 'South-East Asia Region (WHO)', 'OECD Countries', 'Western Pacific Region (WHO)', 'Middle East & North Africa (WB)', 'Eastern Mediterranean Region (WHO)', 'East Asia & Pacific (WB)', 'Region of the Americas (WHO)', 'World Bank Upper Middle Income', 'African Region (WHO)', 'World Bank Low Income', 'Latin America & Caribbean (WB)', 'World Bank High Income', 'Europe & Central Asia (WB)', 'Sub-Saharan Africa (WB)', "Africa", 'More developed regions']

#tolgo i nomi che non sono nomi di nazioni
for i in not_countries:
    N = N[N["Nome"] != i] 

# avendo in certi casi solo dati che vann dal 1990 a 2019 ho deciso di tenere questo arco temporale 
for i in range(1950, 1990):
    N = N[N["Anno"] != i]
for i in range(2020, 2022):
    N = N[N["Anno"] != i]

# faccio in modo che il continente sia visibile per ogni paese durante ogni anno
CONTINENTI = pd.read_csv("../input/continents/country-and-continent-codes-list-csv.csv")
CONTINENTI.columns = ["Continente", "code", "Nome_nazione", "Codice_2","Ccode","Numero"]

# elimino i paesi presenti in due continenti, e facciamo che appartenga solo ad un continente
CONTINENTI = CONTINENTI.drop_duplicates(subset =["Nome_nazione"], keep='last')
CONTINENTI = CONTINENTI[["Continente", "Ccode"]]

# creo la versione inziale della tabella NAZIONE_TEMPO
NAZIONE_TEMPO = pd.merge(CONTINENTI, N, on = "Ccode", how = "inner")
NAZIONE_TEMPO.columns = ["Continente", "Nome", "Ccode", "Anno", "Nome_Area_Geografica"] # per togliere la x dal titolo della colonna dei continenti

# creo la colonna area_geografica
NAZIONE_TEMPO["Nome_Area_Geografica"] = NAZIONE_TEMPO["Continente"]
NAZIONE_TEMPO["Nome_Area_Geografica"] = NAZIONE_TEMPO["Nome_Area_Geografica"].replace(["Asia","Oceania"], "Asia&Oceania")
NAZIONE_TEMPO["Nome_Area_Geografica"] = NAZIONE_TEMPO["Nome_Area_Geografica"].replace(["North America", "South America"], "Americas")
NAZIONE_TEMPO.columns = ["Continente", "Ccode", "Nome", "Anno", "Nome_Area_Geografica"]


# CREAZIONE TABELLA POPOLAZIONE
import pandas as pd

# per creare la tabella popolazione bisogna unire la tabella dell'età media e fertilità media con la tabella dell'aspettativa di vita medio e  il pil procapite
eta_mediana = pd.read_csv("../input/age-structure/median-age-vs-births-per-woman (2).csv")
eta_mediana.columns = ["Nome", "Ccode", "Anno", "Eta_mediana","n_medio_figli_per_donna", "Numero_popolazione", "Continente"]
eta_mediana = eta_mediana.drop([ "Continente"], axis = 1 )

asp_di_vita = pd.read_csv("../input/agestructure/life-expectancy-vs-gdp-per-capita.csv")
asp_di_vita.columns = ["Nome", "Ccode", "Anno", "Aspettativa_di_vita","PIL_procapite","boh","Numero_popolazione", "Continente"]
asp_di_vita = asp_di_vita.drop(["boh","Continente"], axis = 1 )

# unisco le due tabelle 
popolazione_1 = pd.merge(eta_mediana, asp_di_vita, how = "inner")

# alla tabella popolazione aggiungo i dati relativi alla percentuale di popolazione che dipende dai lavoratori
lavoratori = pd.read_csv("../input/lavoratori/dependenxy.csv")
lavoratori.columns = ["Nome", "Ccode", "Anno", "%persone_dipendono_dai_lavoratori"]

# questi sono i dati realtivi alla percentuale di giovani, ossia persone sotto ai 15 anni, che dipende dai lavoratori
giovani= pd.read_csv("../input/ultima-per-popolazione/bebaaaa.csv")
giovani.columns = ["Nome", "Ccode", "Anno", "%bambini_dipendono_dai_lavoratori"]
popolazione_2 = pd.merge(popolazione_1, lavoratori, how = "inner")
popolazione_3 = pd.merge(popolazione_2, giovani, how = "inner")

crescita = pd.read_csv("../input/population-growth/population-growth-rate-vs-median-age (1).csv")
crescita.columns = ["Nome", "Ccode", "Anno", "Rate_di_crescita", "Età_mediana", "Continente"]
crescita = crescita.drop(["Età_mediana", "Continente"], axis = 1)
popolazione_4 = pd.merge(popolazione_3, crescita, how = "inner")

# anche qui tengo solo i dati relativi agli anni 1990-2019
POPOLAZIONE = popolazione_4
for i in range(1950, 1990):
    POPOLAZIONE = POPOLAZIONE[POPOLAZIONE["Anno"] != i]
for i in range(2020, 2022):
    POPOLAZIONE = POPOLAZIONE[POPOLAZIONE["Anno"] != i]

#elimino la colonna Nome 
POPOLAZIONE = POPOLAZIONE.drop(["Nome"], axis = 1)


# CREAZIONE TABELLA FATTORI RILEVANTI

# per la creazione di fattori rilevanti unisco le varie cause di
# morte che influenzano la struttura dell'età in una nazione
# ho deciso di raccogliere i numeri di morti per 100.000 persone di 50 paesi differenti
import pandas as pd

# anche qui elimino i nomi che non erano nomi di nazione 
not_countries = ["Antarctica","Oceania", "Northern America","Low-income countries", "Latin America and the Caribbean"'Least developed countries',"Land-locked Developing Countries (LLDC)","Lower-middle-income countries","Lower-income countries","Small Island Developing States (SIDS)","Western Sahara","Upper-middle-income countries",'Eastern Mediterranean Region (WHO)','Less developed regions, excluding China',"High-income countries","Less developed regions, excluding least developed countries","World","Less developed regions","Two Sicilies","East Germany","North America (WB)","European Region (WHO)", 'World Bank Lower Middle Income', 'South-East Asia Region (WHO)', 'OECD Countries', 'Western Pacific Region (WHO)', 'Middle East & North Africa (WB)', 'Eastern Mediterranean Region (WHO)', 'East Asia & Pacific (WB)', 'Region of the Americas (WHO)', 'World Bank Upper Middle Income', 'African Region (WHO)', 'World Bank Low Income', 'Latin America & Caribbean (WB)', 'World Bank High Income', 'Europe & Central Asia (WB)', 'Sub-Saharan Africa (WB)', "Africa", 'More developed regions','Eastern Mediterranean Region (WHO)', "North America (WB)","European Region (WHO)", 'World Bank Lower Middle Income', 'South-East Asia Region (WHO)', 'OECD Countries', 'Western Pacific Region (WHO)', 'Middle East & North Africa (WB)', 'Eastern Mediterranean Region (WHO)', 'East Asia & Pacific (WB)', 'Region of the Americas (WHO)', 'World Bank Upper Middle Income', 'African Region (WHO)', 'World Bank Low Income', 'Latin America & Caribbean (WB)', 'World Bank High Income', 'Europe & Central Asia (WB)', 'Sub-Saharan Africa (WB)']
set_not_countries = set(not_countries)
set_countries = series.difference(set_not_countries)
series_names = list(set_countries)[50:195]

cancro = pd.read_csv("../input/death-rates/cancer-death-rates.csv")
cancro.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_cancro"]
series = set(cancro["Nome"])

cardio = pd.read_csv("../input/death-rates/cardiovascular-disease-death-rates.csv")
cardio.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_malattie_cardio"]

# unisco le prime due cause di morte in un'unica tabella che chiamo FR1
FR1 = pd.merge(cancro, cardio, how="inner")

# limito il range temporale 
for i in range(1950, 1990):
    FR1 = FR1[FR1["Anno"] != i]
for i in range(2020, 2022):
    FR1 = FR1[FR1["Anno"] != i]

# limito il numero di paesi da analizzare
for i in not_countries:
    FR1 = FR1[FR1["Nome"] != i]
for i in series_names:
    FR1 = FR1[FR1["Nome"] != i]

# proseguo aggiungendo i numeri di morti per 100.000 abitanti di una nazione dovuti ad altre cause  
fumo = pd.read_csv("../input/death-rates/death-rate-smoking.csv")
fumo.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_fumo"]

FR2 = pd.merge(FR1, fumo, how = "inner")

hiv = pd.read_csv("../input/death-rates/hiv-death-rates.csv")
hiv.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_hiv"]
FR3 = pd.merge(FR2, hiv)

fame = pd.read_csv("../input/death-rates/malnutrition-death-rates.csv")
fame.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_fame"]
FR4 = pd.merge(FR3, fame)

men = pd.read_csv("../input/age-structure/men-survival-to-age-65 (1).csv")
men.columns = ["Nome", "Ccode", "Anno", "%Sopravvivenza_u_fino_ai_65"]
FR5 = pd.merge(FR4, men)

women = pd.read_csv("../input/age-structure/women-survival-to-age-65.csv")
women.columns = ["Nome", "Ccode", "Anno", "%Sopravvivenza_d_fino_ai_65"]
FR6 = pd.merge(FR5, women)

suicidio = pd.read_csv("../input/death-rates/suicide-death-rates.csv")
suicidio.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_suicidio"]
FR7 = pd.merge(FR6, suicidio)

distastri_naturali = pd.read_csv("../input/death-rates/death-rates-from-disasters.csv")
distastri_naturali.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_disastri_naturali"]
FR8 = pd.merge(FR7, distastri_naturali)

tubercolosi = pd.read_csv("../input/death-rates/tuberculosis-death-rates.csv")
tubercolosi.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_tubercolosi"]
FR9 = pd.merge(FR8, distastri_naturali)

incidenti_stradali = pd.read_csv("../input/death-rates/death-rates-road-incidents.csv")
incidenti_stradali.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_incidenti_stradali"]
FR10 = pd.merge(FR9, distastri_naturali)

mortalità_infantile = pd.read_csv("../input/child-mortality/child-mortality-igme.csv")
mortalità_infantile.columns = ["Nome", "Ccode", "Anno", "Numero_morti_infantili"]
FR11 = pd.merge(FR10, mortalità_infantile)

demenza = pd.read_csv("../input/death-rates/dementia-death-rates.csv")
demenza.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_demenza_senile"]
FR12 = pd.merge(FR11, demenza)

annegamento = pd.read_csv("../input/death-rates/drowning-death-rates.csv")
annegamento.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_annegamento"]
FR13 = pd.merge(FR12, annegamento)

malaria = pd.read_csv("../input/death-rates/malaria-death-rates.csv")
malaria.columns = ["Nome", "Ccode", "Anno", "Numero_morti_per_malaria"]
FR14 = pd.merge(FR13, malaria)

# tolgo la colonna nome 
FR14 = FR14.drop(["Nome"], axis = 1)

FATTORI_RILEVANTI= FR14

# facciamo la media tra la percentuale di sopravvivenza degli uomini e delle donne fino ai 65 anni 
FATTORI_RILEVANTI['Percentuale_sopravvivenza_fino_65_anni'] = FATTORI_RILEVANTI[['%Sopravvivenza_d_fino_ai_65','%Sopravvivenza_u_fino_ai_65']].mean(axis=1)

# elimino le colonne relative agli uomini e alle donne
FATTORI_RILEVANTI = FATTORI_RILEVANTI.drop(["%Sopravvivenza_d_fino_ai_65","%Sopravvivenza_u_fino_ai_65" ], axis = 1)


# CREAZIONE TABELLA AREA GEOGRAFICA 
# in questa tabella abbiamo il numero di persone morti in conflitti per anno e area geografica, che in alcuni casi non coincide con il continente. 
import pandas as pd 

AREA_GEOGRAFICA = pd.read_csv("../input/conflitti/battle-related-deaths-in-state-based-conflicts-since-1946-by-world-region.csv")
AREA_GEOGRAFICA.columns = ["Nome_Area_Geografica", "Codice", "Anno", "Numero_morti_conflitti"]
AREA_GEOGRAFICA = AREA_GEOGRAFICA.drop(["Codice"], axis = 1 )
continents = set(AREA_GEOGRAFICA["Nome_Area_Geografica"])

for i in range(1946, 1990):
    AREA_GEOGRAFICA = AREA_GEOGRAFICA[AREA_GEOGRAFICA["Anno"] != i]
for i in range(2020, 2022):
    AREA_GEOGRAFICA = AREA_GEOGRAFICA[AREA_GEOGRAFICA["Anno"] != i]

# ho tolto la zona Middle East, siccome le morti in Medio Oriente venogno considerate anche nella sezione di Asia e Oceania, e di Africa nel caso dell'Egitto
for i in continents:
    if i == "World" or i == "Middle East":
        AREA_GEOGRAFICA = AREA_GEOGRAFICA[AREA_GEOGRAFICA["Nome_Area_Geografica"] != i]
        
# CREAZIONE TABELLA
#import os 
#data_folder_2 = "my_csv_output/"
#os.makedirs(data_folder_2)
#FATTORI_RILEVANTI.to_csv(data_folder_2+"FATTORI_RILEVANTI.csv",index=False)
#POPOLAZIONE.to_csv(data_folder_2+"POPOLAZIONE.csv",index=False)
#AREA_GEOGRAFICA.to_csv(data_folder_2+"AREA_GEOGRAFICA.csv",index=False)
#NAZIONE_TEMPO.to_csv(data_folder_2+"NAZIONE_TEMPO.csv",index=False)


#!zip -r my_csv_output.zip my_csv_output